In [8]:
%%time
import math
import torch
import numpy as np

from botorch.test_functions import SixHumpCamel
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.utils.transforms import standardize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import (
    ExpectedImprovement,
    ProbabilityOfImprovement,
    qMaxValueEntropy,
)
from botorch.acquisition.max_value_entropy_search import qLowerBoundMaxValueEntropy
import matplotlib.pyplot as plt
from botorch.optim import optimize_acqf
from botorch.acquisition import qNoisyExpectedImprovement, qProbabilityOfImprovement
from time import time
import cProfile, io, pstats
from pstats import SortKey
import scipy
from scipy.optimize import minimize, Bounds

SMOKE_TEST = True

torch.manual_seed(123456)
torch.set_default_dtype(torch.double)

bounds = torch.tensor(SixHumpCamel._bounds).T
bounds_norm = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(5, 2)
train_Y = SixHumpCamel(negate=True)(train_X).unsqueeze(-1)

train_X = normalize(train_X, bounds=bounds)
train_Y = standardize(train_Y + 0.05 * torch.randn_like(train_Y))

model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll, max_attempts=1)
scipy.__version__

CPU times: user 340 ms, sys: 67.2 ms, total: 407 ms
Wall time: 99.2 ms


'1.15.0'

In [9]:
%%time
print("\n\n\n\n\n\n optimize_acqf \n\n")

acqf = qNoisyExpectedImprovement(model, train_X, cache_root=False)

# eval and maximise acq functions
with cProfile.Profile() as pr:
    optimize_acqf(
        acq_function=acqf,
        bounds=bounds_norm,
        q=5,
        num_restarts=5,
        raw_samples=5,
        sequential=False,
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)

scipy.__version__







 optimize_acqf 




/Users/saitcakmak/botorch/botorch/acquisition/monte_carlo.py:496: NumericsWarning: qNoisyExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qNoisyExpectedImprovement 	 --> 	 qLogNoisyExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)


         630577 function calls (601743 primitive calls) in 1.008 seconds

   Ordered by: cumulative time
   List reduced from 550 to 55 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.008    1.008 /Users/saitcakmak/botorch/botorch/optim/optimize.py:479(optimize_acqf)
        1    0.000    0.000    1.008    1.008 /Users/saitcakmak/botorch/botorch/optim/optimize.py:630(_optimize_acqf)
        1    0.000    0.000    1.008    1.008 /Users/saitcakmak/botorch/botorch/optim/optimize.py:310(_optimize_acqf_batch)
        1    0.000    0.000    0.998    0.998 /Users/saitcakmak/botorch/botorch/optim/optimize.py:353(_optimize_batch_candidates)
        1    0.000    0.000    0.998    0.998 /Users/saitcakmak/botorch/botorch/generation/gen.py:44(gen_candidates_scipy)
        1    0.000    0.000    0.990    0.990 /Users/saitcakmak/botorch/botorch/optim/utils/timeout.py:19(minimize_with_timeout)
        1    0.000    0.0

'1.15.0'

In [10]:
%%time
print("\n\n\n\n\n\n calling minimize directly \n\n")


def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((5, 5, 2))).contiguous().requires_grad_(True)
    loss = acqf(X_tensor).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

bounds = Bounds(
    lb=np.zeros(50),
    ub=np.ones(50),
    keep_feasible=True,
)
x0 = np.random.random(50)

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)

scipy.__version__







 calling minimize directly 


         508933 function calls (485525 primitive calls) in 0.825 seconds

   Ordered by: cumulative time
   List reduced from 492 to 49 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.825    0.825 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_minimize.py:53(minimize)
        1    0.001    0.001    0.825    0.825 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:290(_minimize_lbfgsb)
      352    0.002    0.000    0.817    0.002 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:71(_compute_if_needed)
      181    0.000    0.000    0.816    0.005 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:293(_update_fun)
      176    0.000    0.000    0.816    0.005 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optim

/opt/anaconda3/envs/botorch/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


'1.15.0'

In [11]:
%%time
print("\n\n\n\n\n\n 100 acqf evaluations \n\n")

with cProfile.Profile() as pr:
    for _ in range(1):
        test_X = torch.rand(20, 5, 2, requires_grad=True)
        acqf(test_X).sum().backward()

# sortby = SortKey.CUMULATIVE
# ps = pstats.Stats(pr).sort_stats(sortby)
# ps.print_stats(.1)

scipy.__version__







 100 acqf evaluations 


CPU times: user 32.4 ms, sys: 8.85 ms, total: 41.2 ms
Wall time: 69.5 ms


'1.15.0'